# 4. IAM 

Use IAM/Credential Report to see overview of all users and their access.
For each user go to Access Advisor and see what are they actually using.
Do not use root account for daily stuff
One physical user - one AWS user
Create roles for AWS services
Assign users to Security Groups
Never share IAM users and Access Keys

# 5. EC2

EC2 User Data script can bootstrap the instance. It is executed only once when the instance is launched.

Example: m5.xlarge: m is instance class, 5 is the generation, xlarge is the size.

- `c` class: compute optimized
- `R`, `X`, `High Memory`, `z1d` type: RAM optimized, for in memory databases, cache, real time big data
- 'I', 'D', 'H1': storage intensive (databases, cache, etc.)
- and many more: [ec2instances info](https://instances.vantage.sh)

####  Security Groups

- security group is a firewall, regulate ports, IP ranges, inbound and outbound traffic
- 0.0.0.0/0 means all IP addresses
- can be attached to multiple instances
- locekd to the region
- lives outside the instance
- **good to maintain a security group for SSH access**
- any timeout is security group issue
- connection refused is instance issue
- by default all inbound is closed, all outbound is open
- security group can reference another security group, this is good to allow bunch of instances to talk to - each other in groups

- 22 is SSH
- 21 is FTP
- 80 is HTTP unsecured websites
- 443 is HTTPS secured websites
- 3389 is RDP (for Windows)


If you lose Key pair pem file you will need to create a new public one from it like:
```bash
ssh-keygen -y -f Test.pem > Test.pub
cat Test.pub
```
then connect to the instance using browser console and add the public key to the authorized_keys file:
```bash
nano ~/.ssh/authorized_keys
```
with that you can connect to the instance from local machine::
```bash
chmod 400 Test.pem  # to change the permissions of the key
ssh -i Test.pem ubuntu@<public IP>  # for for Ubuntu and for Amazon Linux AMI use `ec2-user`
```

Always attach roles to the instance to give access to other AWS services.

- On-demand, Reserved, Spot, and Dedicated Hosts are the pricing models for EC2 instances.




# 6. Instance Storage

#### EBS (Elastic Block Storage)

They are network drives you can attach to your instances while they run. They can persist data after the instance is terminated. They are bound to a specific availability zone. They are automatically replicated within the availability zone. Can be attached only to one instance at a time.

What if we want to transfer Volume to different AZ or to simply back it up, then we have to snapshot it. We can then recreate a new EBS from the snapshot, even in a different AZ. We can also archive the snapshot but it takes 24-48 hours to get it back.

#### AMI (Amazon Machine Image)

AMI is super useful to speed up the new instance launching, it is equivalent to run bunch of user data and then store the state of the machine.

#### EC2 Image Builder

This is service that can build and test AMIs. One can first create Components, then bundle them to create AMIs. These AMIs can also be tested automatically, updates scheduled, etc.

#### EC2 Instance Store

It is a temporary block storage for EC2 instances. It is lost when the instance is stopped or terminated. It is physically attached to the machine hosting the instance. It is much faster than EBS but it's available only on certain instance types (for example `g4dn`, `i3`, `m5d`, etc. but not on small `t` types).

#### EFS Elastic File System

This is a managed NFS (Network File System) that can be mounted on many EC2 instances. It is highly available, scalable. It can be used for content management, web serving, data sharing, etc. It is 2-3X more expensive than EBS. Available in all AZs at the same time. EFS Infrequent Access (EFS-IA) is a cheaper/slower version of EFS with lifecycle policy.

#### Amazon FSx for 1) Lustre (Linux + Cluster = Lustre) and 2) Windows File Server

This is a high-performance file system optimized for fast processing of workloads such as machine learning, high-performance computing, video processing, financial modeling, etc. It is a fully managed service that can be integrated with S3. The data is stored in S3 but managed by AWS with super small latencies.




 
# 7. Load Balancers

There is Application (HTTP like, layer 7), Network (high performance, millions request per second, TCP,UDP,TLS, layer 4), and Gateway (IP packets themselves, GENEVE protocol, layer 3, typically used to check security via 3rd party).

We run 2 instances, join them in a target group, and then create an Application LB (ALB) and point to the group. The LB is managed for us.


## Autoscaling group (ASG)

ASG embodies elasticity, across multiple AZ. There is minimum, desired, and maximum number of instances. We need to set up template, type, security group, EBS, the usual. The ASG will now always try to keep desired number of instances even if they crash. There are scaling options (Manual, Dynamic(Simple, Target, Schedule), and Predictive).














# 8. S3

S3 has 11 nines durability and 4 nines availability.

Replication works only with versioning enabled. It has to be enabled in the bucket tab Management -> Replication. 
There is same region replication (SSR) and cross region replication (CRR). It is asynchronous and can be used for disaster recovery, backups, etc.


There are several storage classes from "S3 Express One Zone" via "Standard" to "Glacier". We can define these when uploading the file or later in the Management tab. We can also set up lifecycle policies to move files between these classes (say move Standard to Intelligent Tiering after 30 days, then to Glacier after 60 days, etc.).

Snow Family is a physical device that can be used to transfer data to AWS. It is secure, fast, and can be used for large data transfers. SnowCone and SnowBall are the most popular devices.
SSD (Solid State Drive) is more advanced then HDD (Hard Disk Drive).

# 9. Databases

RDS is relational store, can create snapshot to back up or replicate, can create read replicas.

Elastic Cache is in-memory data store, can be used for caching, session store, etc. Can be used on many database types.

Dynamo DB is a NoSQL, it's serverless (for us) and single digit millisecond latency. Stores key-value pairs, documents, graphs, etc.

DynamoDB Accelerator (DAX) is a caching layer in front of DynamoDB, and no other database. It's 10X better then ElastiCache.

Active-active replication means user can write in both regions.




# 10. ECS, Lambda, Batch, Lightsail

ECS and Fargate can run containers on AWS. ECS is more control, Fargate is serverless. ECS has Task Definition, Service, Cluster. Fargate has only Task Definition.

Lightsail is miniature AWS, has fixed prices and trimmed down flexibility.

AWS Batch run processes in batch.




# 11. Deployment

- CloudFormation is Infra as Code,
- Beanstalk (PaaS)
    - Role needs AWSElasticBeanstalkWebTier, AWSElasticBeanstalkMulticontainerDocker, AWSElasticBeanstalkWorkerTier
    - Beanstalk uses CloudFormation
- CodeDeploy (Hybrid)
- AWS System Manager (SSM) (hybrid) is a service that allows you to automate tasks across your AWS resources. It can be used to patch, automate, run commands, etc.
It needs IAM role that allows SSM to be assigned to EC2 instance. It also needs SSM agent installed but this should be there by default on Amazon AMIs and Ubuntu (you will need to install it for on-premises machines).
So now we can connect to EC2 machine without SSH port open. One can find all the instances in the Systems Manager -> Fleet Manager.
- AWS Parameter Store is a service that allows you to store configuration data and secrets. It can be used to store passwords, database strings, etc. It can be used with IAM roles to give access to the instances. 


- Code Commit is a Git service
- Code Build is a build service
- Code Deploy is a deployment service
- Code Pipeline is a orchestration service
- CodeStart is allowing for CICD
- Cloud9 is an IDE in the cloud
- AWS CDK (this is creating services using some language like Python, which is then translated to CloudFormation)






















# 12. Global services

### Route 53 - managed DNS
- hostname to IPv4 is called A-record
- hostname to IPv6 is called AAAA record
- hostname to hostname is called CNAME
- hostname to AWS resource is called Alias

Simple routing policy is one record with multiple IP addresses
Weighted routing policy is to distribute the traffic
Latency routing policy is to minimize latency
Failover routing policy is to help with disaster recovery

We can buy a domain, then create EC2 web servers that host some application, and then create a Route 53 A-records to point to the EC2 web servers, say based on latency. Then, when a user, from around the world, goes to the domain, they will be redirected to the closest server.

### AWS CloudFront 

AWS CloudFront is CDN and it caches content around the world. Origin can be S3 bucket or HTTP backend, or anything. 
Enhanced security with CloudFront Origin Access Control (OAC).
Note that CloudFront is great for static content, S3 cross-region-replication is great for dynamic content that need to have low-latency.

We first create a CloudFront distribution, then we point it to the S3 bucket, then we can set up the cache behavior, TTL, etc. We can also set up the security, like Origin Access Control (OAC) to restrict access to the S3 bucket, we need to S3 bucket a policy so it is accessible only from CloudFront.

Web application firewall (WAF) and AWS Shield are CloudFront security services.

### S3 Accelerated Transfer 
S3 buckets are linked to only one region. If you want to upload to a S3 bucket that is far away, you want to use S3 Accelerated Transfer (this is just a feature under bucket/properties). [This](http://s3-accelerate-speedtest.s3-accelerate.amazonaws.com/en/accelerate-speed-comparsion.html) tool compares S3 transfer acceleration upload/download to one specific bucket. So if you need fast transfer to S3 bucket from around the world, use it (btw, the acceleration happens by using edge locations).


### AWS Global accelerator
AWS Global accelerator, similarly, leverages internal AWS network (which is fast) to optimize route to the application using edge locations (note that Global accelerator does not cache any content). If enabled, 2 Anycast IP addresses are assigned to the application, and the traffic is routed to the optimal endpoint. [Test](https://speedtest.globalaccelerator.aws/#/). For global accelerator we need to create 1) accelerator, 2) listener, 3) endpoint group, 4) endpoints. We can also set up the traffic dials, like percentage of traffic to go to one endpoint, etc.

AWS Outpost, is like having AWS racks on-premisses.

AWS Wavelength, is like having basic AWS racks in the 5G towers (with EC2 instances, EBS, etc.) for ultra-low latency applications. Again, 5G only.

AWS Local Zones extend VPC into more zones outside default AZs. For example if we define a subnet in Boston, we can then create EC2 instance in that zone. This allows ultra-low latency applications.

Final notes on global services:
- Single-region, single AZ is easy to set up, but not fault tolerant. 
- Single-region, multi-AZ is fault tolerant, but not disaster tolerant.
- Multi-region Active-Passive, only writes to active. Disaster tolerant, but more expensive and complex.
- Multi-region Active-Active, writes to both. Disaster tolerant, but more expensive and complex.

# 13. Cloud Integration

Let's decouple services, so that they can be scaled independently. We can use SQS, SNS, Kinesis, and Amazon MQ for this.

### SQS (Simple Queue Service)

Producer/Consumer model. Pull based. 1-10K per second. Message retention is 4-14 days. Standard or FIFO order.

### Amazon Kinesis

Kinesis = real-time big data. Gets data (Kinesis Data Streams), analytics using SQL (Kinesis Data Analytics), save into S3, Redshift (Kinesis Firehose). Side thing would be real-time video streams (Kinesis Video Streams).

### SNS (Simple Notification Service)

Push based notifications to mobile devices, email, HTTP endpoints, etc. It is pub/sub model. It can be used to send messages to multiple subscribers. Mailinator.com is a good tool to test email notifications.

### Amazon MQ

SQS and SNS are cloud native, Amazon MQ is managed message broker service for ActiveMQ and RabbitMQ. It is good for legacy applications that use ActiveMQ or RabbitMQ. Amazon MQ doesn't scale as much as SQS. 

# 14. Monitoring

EventBridge can be reactive to any event or schedule to send emails, run Lambdas etc. It can also accepts third party events. It is quite powerful. It's also serverless.


# 17. Machine Learning

- Rekognition is for classification, detection, recognition, etc. of images and videos.
- Transcribe is for speech to text. Can redact, works on multiple languages. Can create metadata for searchable archive. This can be useful for podcasts. 
- Polly - text to speech.
- Lex - chatbots, bundled with Connect but can trigger Lambda functions.
- Comprehend - NLP, sentiment analysis, key phrase extraction, language detection, etc.
- SageMaker - ML service, can train, tune, deploy models.
- Translate - language translation.
- Forecast - time series forecasting.
- Kendra - document search service.
- Personalize - recommendation service.
- Textract - OCR, can extract text, tables, etc. from images.


